<a href="https://colab.research.google.com/github/KenYew/sre-notes/blob/main/networking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌍 Networking

---
## What happens if I type example.com in my browser and hit enter?

### Step 1: 📖 DNS
1. Checks if the DNS entry is available in any of the below cache layers.
  * Browser
  * Operating System
  * Local Area Network (Router)
  * DNS Server Cache (Internet Service Provider or Corporate Network)
2. If there is a DNS cache miss, the DNS resolver of the DNS server (from ISP or corporate network) performs a recursive DNS lookup.
3. DNS resolver recursively queries for the DNS answer from:
  * Root name server
  * Top level domain (TLD) name server
  * Route 53 DNS name server
4. DNS server returns the DNS answer (IP address of example.com) to the web browser and caches it. 
5. Web browser uses this resolved IP address to send requests to the web server of example.com
6. Web server of example.com returns the webpage to the web browser and displays the webpage.

### Step 2a: ⚡️ TCP

1. Packets from the client browser's request will be routed from the router, ISP and then to the Internet Exchange (switching between ISPs or networks) in TCP to find the server with the IP address to connect to.
1. When the browser finds the server on the Internet, it establishes a TCP connection with the server using a 3-way handshake.
1. If HTTPS is being used, a TLS handshake is performed to secure the communication.

#### 🤝 TCP 3-Way Handshake

<img src="https://drive.google.com/uc?id=1s33vrvQDL4pFy4BflSBU8VwJecHo4ddB" align="middle" width="500px"/>



This send and receive happens multiple times following the TCP connection flow:

1. Client chooses an initial sequence number (ISN) and sends the packet to the server with the SYN bit set to indicate it is setting the ISN

1. Server receives SYN and if it's in an agreeable mood:
    * Server chooses its own initial sequence number
    * Server sets SYN to indicate it is choosing its ISN
    * Server copies the (client ISN +1) to its ACK field and adds the ACK flag to indicate it is acknowledging receipt of the first packet
1. Client acknowledges the connection by sending a packet:
    * Increases its own sequence number
    * Increases the receiver acknowledgment number
    * Sets ACK field
1. Data is transferred as follows:
    * As one side sends N data bytes, it increases its SEQ by that number
    * When the other side acknowledges receipt of that packet (or a string of packets), it sends an ACK packet with the ACK value equal to the last received sequence from the other
1. To close the connection:
    * The closer sends a FIN packet
    * The other sides ACKs the FIN packet and sends its own FIN
    * The closer acknowledges the other side's FIN with an ACK

#### 🔐 TLS Handshake

<img src="https://drive.google.com/uc?id=1Qm3Te9rH-piAjkyWUQOh7J39QMzhQTl6" align="middle" width="500px"/>

1. The client computer sends a `ClientHello` message to the server with its Transport Layer Security (TLS) version, list of cipher algorithms and compression methods available.
1. The server replies with a `ServerHello` message to the client with the TLS version, selected cipher, selected compression methods and the server's public certificate signed by a CA (Certificate Authority). The certificate contains a public key that will be used by the client to encrypt the rest of the handshake until a symmetric key can be agreed upon.
1. The client verifies the server digital certificate against its list of trusted CAs. If trust can be established based on the CA, the client generates a string of pseudo-random bytes and encrypts this with the server's public key. These random bytes can be used to determine the symmetric key.
1. The server decrypts the random bytes using its private key and uses these bytes to generate its own copy of the symmetric master key.
1. The client sends a `Finished` message to the server, encrypting a hash of the transmission up to this point with the symmetric key.
1. The server generates its own hash, and then decrypts the client-sent hash to verify that it matches. If it does, it sends its own `Finished` message to the client, also encrypted with the symmetric key.
1. From now on the TLS session transmits the application (HTTP) data encrypted with the agreed symmetric key.

#### 👮 TCP Congestion Control

* Sometimes, due to network congestion or flaky hardware connections, TLS packets will be dropped before they get to their final destination. 
* The sender then has to decide how to react. The algorithm for this is called TCP congestion control. 

    1. Client chooses a congestion window based on the maximum segment size (MSS) of the connection.
    1. For each packet acknowledged, the window doubles in size until it reaches the 'slow-start threshold'. In some implementations, this threshold is adaptive.
    1. After reaching the slow-start threshold, the window increases additively for each packet acknowledged. If a packet is dropped, the window reduces exponentially until another packet is acknowledged.

### Step 2b: 🌍 CDN

1. The CDN caches static and dynamic content closer to the browser using edge locations. 
2. It is a global distributed network of caching servers to improve performance of a website by bringing content closer to your users. 
3. Instead of relying on the public internet routing infrastructure (subject to extra hops, redeliveries and packet loss), client browser request gets to ride on the AWS Global Network. 
4. Request is intelligently routed through the most performant location to deliver content to your browser.

### Step 3: 💻 HTTP

1. When the browser has an established TCP connection to the server, it follows the rules of communication of HTTP. 
2. Browser sends an HTTP request to the server to request contents of a page. 
3. HTTP request consists of request line, headers (metadata of the request) and body. 
    * Request line contains the following:
        * a request method, which is one of `GET, POST, PUT, PATCH, DELETE`, or a handful of other HTTP verbs
        * the path, pointing to the requested resources
        * the HTTP version to communicate with

    * Request headers pass extra information along from client that help route the request and indicate what type of client is making (user agent). 
    ```
    Host: jennapederson.dev
    User-Agent: curl/7.64.1
    Accept: */*
    ```
    * Body is usually empty for `GET` but for `POST, PUT or PATCH`, the body will contain data from the client to create or update. It can be of different formats and the server understands the format based on the request header, `Content-Type`. 
4. Once the server has received the request from the client, the server processes it and sends back a response. 
5. Response 200 Status code is received by the browser and renders the response. 
6. Browser inspects the response headers and renders the response.
    * Example: If `Content-Type` is HTML, browser knows how to render the resource in HTML. It may make additional requests to get JavaScript, CSS, images and data.